<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Aus_Inclinometro_GKN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de archivos GKN de Inclinómetros en Google Colab

Este notebook está dividido en 3 pasos principales para procesar archivos GKN (inclinómetros), desde la carga hasta la exportación de resultados.  
Soporta archivos individuales, carpetas, archivos comprimidos (ZIP) y almacenamiento local, en `/content` o en Google Drive.

## Paso 1: Definir funciones de procesamiento

Define las funciones para procesar archivos `.gkn` y combinarlos en un solo DataFrame.  
No necesitas modificar nada aquí, solo ejecuta la celda.

In [31]:
import pandas as pd
import os

def procesar_gkn_inclinometros(file_path):
    try:
        print(f"Procesando el archivo: {file_path}")
        with open(file_path, 'r') as file:
            lines = file.readlines()
            fecha = next((line.split(":")[1].strip() for line in lines if line.startswith("DATE")), None)
            nro_sonda = next((line.split(":")[1].strip() for line in lines if line.startswith("PROBE NO.")), None)
            hole_no = next((line.split(":")[1].strip() for line in lines if line.startswith("HOLE NO.")), None)
            start_idx = next((i + 2 for i, line in enumerate(lines) if '#READINGS:' in line), None)
            if start_idx is None:
                raise ValueError(f"No se encontraron lecturas en el archivo {file_path}")
            data = [line.strip().split(',') for line in lines[start_idx:]]
            df = pd.DataFrame(data, columns=['Profundidad', 'A+', 'A-', 'B+', 'B-'])
            df = df.apply(pd.to_numeric, errors='coerce')
            df['Fecha'] = pd.to_datetime(fecha, dayfirst=True, errors='coerce').strftime('%d/%m/%Y')
            df['Inclinómetro'] = hole_no
            df['Nro. Sonda'] = nro_sonda
            df = df[['Fecha', 'Inclinómetro', 'Profundidad', 'A+', 'A-', 'B+', 'B-', 'Nro. Sonda']]
            print(f"Archivo procesado correctamente: {file_path}")
            return df
    except Exception as e:
        print(f"Error procesando el archivo {file_path}: {e}")
        return None

def procesar_varios_gkn(file_list):
    dfs = []
    for file_path in file_list:
        df = procesar_gkn_inclinometros(file_path)
        if df is not None:
            dfs.append(df)
    if dfs:
        df_final = pd.concat(dfs, ignore_index=True)
        print("Todos los archivos fueron procesados correctamente.")
        return df_final
    else:
        print("No se pudieron procesar los archivos.")
        return None

## Paso 2: Selección y carga de archivos

Puedes elegir una de las siguientes opciones como fuente de tus archivos GKN:
- **1:** Subir un archivo ZIP desde tu PC (que contenga todos los `.gkn`).
- **2:** Procesar una carpeta en Google Drive.
- **3:** Procesar una carpeta en `/content` de Colab.
- **4:** Seleccionar múltiples archivos desde tu PC.

Los archivos encontrados se procesarán automáticamente.


In [32]:
import os
from google.colab import files

def seleccionar_fuente_gkn():
    print("Selecciona la fuente de tus archivos GKN:")
    print("1 - Subir ZIP con todos los archivos desde tu computadora")
    print("2 - Carpeta en Google Drive")
    print("3 - Carpeta en /content de Colab")
    print("4 - Seleccionar carpeta localmente (visual, solo en Colab)")
    fuente = input("Escribe 1, 2, 3 o 4 y presiona ENTER: ")
    return fuente.strip()

file_names = []
fuente = seleccionar_fuente_gkn()

if fuente == "1":
    print("Sube el archivo .zip que contiene todos los archivos GKN:")
    uploaded = files.upload()
    import zipfile
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('GKN_folder')
            print("ZIP descomprimido en /content/GKN_folder")
        else:
            print("El archivo subido no es un .zip")
    carpeta_gkn = '/content/GKN_folder'
    file_names = [os.path.join(carpeta_gkn, f) for f in os.listdir(carpeta_gkn) if f.lower().endswith('.gkn')]
    print("Archivos GKN encontrados:", file_names)

elif fuente == "2":
    from google.colab import drive
    drive.mount('/content/drive')
    carpeta_gkn = input("Pega la ruta completa de la carpeta en Google Drive (ej: /content/drive/MyDrive/MisGKN): ").strip()
    if not os.path.exists(carpeta_gkn):
        raise ValueError("La carpeta especificada no existe.")
    file_names = [os.path.join(carpeta_gkn, f) for f in os.listdir(carpeta_gkn) if f.lower().endswith('.gkn')]
    print("Archivos GKN encontrados:", file_names)

elif fuente == "3":
    carpeta_gkn = input("Pega la ruta de la carpeta en /content (ej: /content/GKN): ").strip()
    if not os.path.exists(carpeta_gkn):
        raise ValueError("La carpeta especificada no existe.")
    file_names = [os.path.join(carpeta_gkn, f) for f in os.listdir(carpeta_gkn) if f.lower().endswith('.gkn')]
    print("Archivos GKN encontrados:", file_names)

elif fuente == "4":
    try:
        from google.colab import widgets
        import shutil
        print("Selecciona la carpeta con los archivos GKN en tu computadora.")
        # Usar el selector visual
        folder_uploaded = files.upload()
        local_folder = '/content/selected_folder'
        os.makedirs(local_folder, exist_ok=True)
        # Mueve los archivos subidos a una carpeta
        for filename in folder_uploaded.keys():
            shutil.move(filename, os.path.join(local_folder, filename))
        file_names = [os.path.join(local_folder, f) for f in os.listdir(local_folder) if f.lower().endswith('.gkn')]
        print("Archivos GKN encontrados:", file_names)
    except Exception as e:
        print("Funcionalidad visual de carpeta local no disponible. Usa ZIP o Drive para carpetas masivas.")
        file_names = []

else:
    print("Opción no válida. Ejecuta la celda de nuevo y elige 1, 2, 3 o 4.")

# Procesar todos los archivos encontrados (si hay)
if file_names:
    df_gkn = procesar_varios_gkn(file_names)
    if df_gkn is not None:
        display(df_gkn.head())
else:
    print("No se encontraron archivos para procesar.")

Selecciona la fuente de tus archivos GKN:
1 - Subir ZIP con todos los archivos desde tu computadora
2 - Carpeta en Google Drive
3 - Carpeta en /content de Colab
4 - Seleccionar carpeta localmente (visual, solo en Colab)


KeyboardInterrupt: Interrupted by user

## Paso 3: Guardar resultados

Por cada nombre de inclinómetro encontrado, se generan dos archivos:  
- Un archivo `.csv`
- Un archivo `.xlsx`

Puedes elegir dónde guardar estos archivos:
- Descargar a tu computadora.
- Guardar en `/content` de Colab.
- Guardar en una carpeta de tu Google Drive.


In [ ]:
from google.colab import files
import os

print("¿Dónde quieres guardar los archivos?")
print("1 - Descargar a mi computadora (PC)")
print("2 - Guardar en /content de Colab")
print("3 - Guardar en mi Google Drive")
opcion = input("Elige 1, 2 o 3: ").strip()

if 'df_gkn' in locals() and df_gkn is not None and not df_gkn.empty:
    nombres_inclinometro = df_gkn['Inclinómetro'].unique()
    print(f"Se encontraron los siguientes inclinómetros: {nombres_inclinometro}")
    rutas_guardadas = []
    for nombre in nombres_inclinometro:
        df_sel = df_gkn[df_gkn['Inclinómetro'] == nombre]
        base_nombre = f"inclinometro_{str(nombre).replace(' ','_').replace('/','-')}"
        archivo_xlsx = f"{base_nombre}.xlsx"
        archivo_csv = f"{base_nombre}.csv"

        if opcion == "1":
            # Guardar temporalmente en /content y descargar
            df_sel.to_excel(archivo_xlsx, index=False)
            df_sel.to_csv(archivo_csv, index=False)
            print(f"Descargando: {archivo_xlsx} y {archivo_csv}")
            files.download(archivo_xlsx)
            files.download(archivo_csv)
            rutas_guardadas.extend([archivo_xlsx, archivo_csv])

        elif opcion == "2":
            # Guardar solo en /content
            df_sel.to_excel(f"/content/{archivo_xlsx}", index=False)
            df_sel.to_csv(f"/content/{archivo_csv}", index=False)
            print(f"Guardado en /content: {archivo_xlsx} y {archivo_csv}")
            rutas_guardadas.extend([f"/content/{archivo_xlsx}", f"/content/{archivo_csv}"])

        elif opcion == "3":
            # Guardar en Google Drive (asegúrate de tener Drive montado)
            from google.colab import drive
            drive.mount('/content/drive', force_remount=True)
            ruta_drive = input("Escribe la ruta completa en tu Drive donde guardar los archivos (ej: /content/drive/MyDrive/MisGKN): ").strip()
            os.makedirs(ruta_drive, exist_ok=True)
            ruta_xlsx = os.path.join(ruta_drive, archivo_xlsx)
            ruta_csv = os.path.join(ruta_drive, archivo_csv)
            df_sel.to_excel(ruta_xlsx, index=False)
            df_sel.to_csv(ruta_csv, index=False)
            print(f"Guardado en Drive: {ruta_xlsx} y {ruta_csv}")
            rutas_guardadas.extend([ruta_xlsx, ruta_csv])

        else:
            print("Opción no válida.")
            break

    print("Archivos generados:", rutas_guardadas)
else:
    print("No hay datos procesados para guardar.")

# Paso 4: ETL y Análisis Exploratorio de Datos de Inclinómetros

Este paso realiza un análisis ETL típico sobre el DataFrame procesado:

- **Extracción:** Usa el DataFrame obtenido previamente.
- **Transformación:**
  - Elimina duplicados y nulos.
  - Convierte tipos de datos.
  - Genera estadísticas descriptivas.
  - Normaliza variables de lectura.
  - Detecta valores atípicos (outliers) usando Z-score.
  - Agrupa por inclinómetro y fecha para obtener resúmenes.
- **Carga:** Exporta el DataFrame transformado a CSV y Excel.

Puedes adaptar los pasos de transformación según tu análisis específico (por ejemplo, agregar análisis de tendencias, visualizaciones, etc.).

---

In [ ]:
# PASO 4: Análisis ETL completo sobre el DataFrame de inclinómetros
#
# Este bloque realiza un proceso ETL típico:
# - Extracción: Ya obtuviste el DataFrame df_gkn en pasos anteriores.
# - Transformación: Limpieza, tipos de datos, normalización, análisis estadísticos y detección de outliers.
# - Carga: Visualización y exportación del resultado transformado.
#
# Puedes modificar los pasos según tu necesidad.

import pandas as pd
import numpy as np

# --- EXTRACT ---
# df_gkn ya debería estar cargado. Si no, carga tu archivo aquí.
# df_gkn = pd.read_excel('mi_archivo.xlsx')  # Descomenta si necesitas cargar manualmente

# --- TRANSFORM ---

print("1. Información general del DataFrame:")
print(df_gkn.info())
display(df_gkn.head())

print("\n2. Eliminar duplicados...")
df_gkn = df_gkn.drop_duplicates()
print("Filas tras eliminar duplicados:", len(df_gkn))

print("\n3. Comprobar valores nulos por columna:")
print(df_gkn.isnull().sum())

print("\n4. Rellenar o eliminar nulos...")
# Ejemplo: rellenar nulos en 'Nro. Sonda' con 'Desconocido'
df_gkn['Nro. Sonda'] = df_gkn['Nro. Sonda'].fillna('Desconocido')
# Opcional: eliminar filas con muchos nulos
df_gkn = df_gkn.dropna(subset=['Profundidad', 'A+', 'A-', 'B+', 'B-'])

print("\n5. Conversión de tipos:")
df_gkn['Fecha'] = pd.to_datetime(df_gkn['Fecha'], dayfirst=True, errors='coerce')
for col in ['Profundidad', 'A+', 'A-', 'B+', 'B-']:
    df_gkn[col] = pd.to_numeric(df_gkn[col], errors='coerce')

print("\n6. Resumen estadístico:")
display(df_gkn.describe())

print("\n7. Normalización (opcional, crea columnas '_norm'):")
for col in ['A+', 'A-', 'B+', 'B-']:
    col_norm = col + '_norm'
    df_gkn[col_norm] = (df_gkn[col] - df_gkn[col].mean()) / df_gkn[col].std()

print("\n8. Detección de valores atípicos (outliers) en A+, A-, B+, B- (Z-score > 3):")
for col in ['A+', 'A-', 'B+', 'B-']:
    z_score = np.abs((df_gkn[col] - df_gkn[col].mean()) / df_gkn[col].std())
    outliers = df_gkn[z_score > 3]
    print(f"{col}: {len(outliers)} outliers")
    if not outliers.empty:
        display(outliers[['Fecha', 'Inclinómetro', 'Profundidad', col]])

print("\n9. Agrupar por inclinómetro y fecha, obtener estadísticas:")
agrupado = df_gkn.groupby(['Inclinómetro', 'Fecha']).agg(
    {'Profundidad':'count', 'A+':'mean', 'A-':'mean', 'B+':'mean', 'B-':'mean'}
).rename(columns={'Profundidad':'Lecturas'}).reset_index()
display(agrupado)

print("\n10. Guardar el DataFrame transformado (opcional):")
df_gkn.to_csv('gkn_etl_transformado.csv', index=False)
df_gkn.to_excel('gkn_etl_transformado.xlsx', index=False)
print("Archivos guardados: gkn_etl_transformado.csv y gkn_etl_transformado.xlsx")

# Puedes descargar con:
# from google.colab import files
# files.download('gkn_etl_transformado.csv')
# files.download('gkn_etl_transformado.xlsx')

In [ ]:
# PASO 5: Visualizaciones exploratorias del DataFrame de inclinómetros
#
# Esta celda genera gráficos para analizar visualmente el comportamiento de los datos.
# Puedes agregar o modificar visualizaciones según tu necesidad.

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Asegura que la fecha sea tipo datetime
df_gkn['Fecha'] = pd.to_datetime(df_gkn['Fecha'], errors='coerce')

# 1. Distribución de profundidades medidas
plt.figure(figsize=(8,4))
sns.histplot(df_gkn['Profundidad'], bins=30, kde=True)
plt.title('Distribución de Profundidad')
plt.xlabel('Profundidad')
plt.ylabel('Frecuencia')
plt.show()

# 2. Evolución temporal de A+ por inclinómetro
plt.figure(figsize=(12,6))
for nombre in df_gkn['Inclinómetro'].unique():
    datos = df_gkn[df_gkn['Inclinómetro'] == nombre]
    datos_agr = datos.groupby('Fecha')['A+'].mean().reset_index()
    plt.plot(datos_agr['Fecha'], datos_agr['A+'], marker='o', label=str(nombre))
plt.title('Evolución temporal de A+ por Inclinómetro')
plt.xlabel('Fecha')
plt.ylabel('A+ promedio')
plt.legend()
plt.show()

# 3. Boxplot de lecturas para detectar outliers por sensor y canal
plt.figure(figsize=(10,6))
sns.boxplot(x='Inclinómetro', y='A+', data=df_gkn)
plt.title('Boxplot de A+ por Inclinómetro')
plt.show()

plt.figure(figsize=(10,6))
sns.boxplot(x='Inclinómetro', y='B+', data=df_gkn)
plt.title('Boxplot de B+ por Inclinómetro')
plt.show()

# 4. Mapa de calor de correlación entre canales
plt.figure(figsize=(8,6))
sns.heatmap(df_gkn[['A+','A-','B+','B-']].corr(), annot=True, cmap='coolwarm')
plt.title('Mapa de calor de correlación entre lecturas')
plt.show()

# 5. Lecturas en función de la profundidad para un inclinómetro seleccionado
inclino = df_gkn['Inclinómetro'].unique()[0]
df_filtro = df_gkn[df_gkn['Inclinómetro'] == inclino]
plt.figure(figsize=(10,6))
sns.scatterplot(x='Profundidad', y='A+', data=df_filtro, hue='Fecha', palette='viridis')
plt.title(f'Lectura A+ vs Profundidad para {inclino}')
plt.xlabel('Profundidad')
plt.ylabel('A+')
plt.legend(title='Fecha', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Puedes agregar más gráficos (tendencias, series de tiempo, etc.) según tus necesidades.

In [ ]:
# PASO 6: Visualizaciones interactivas de inclinómetros con Plotly (versión automática)
#
# Esta celda genera gráficos interactivos de todas las variables para cada inclinómetro,
# por profundidad y fecha, SIN pedir el nombre ni guardar los archivos (solo los muestra en pantalla).
# Puedes modificarla para guardar más adelante si lo deseas.

import plotly.express as px
import pandas as pd

# Asegura que la fecha sea tipo datetime
df_gkn['Fecha'] = pd.to_datetime(df_gkn['Fecha'], errors='coerce')

# 1. Para cada inclinómetro, genera los gráficos interactivos de todas las variables
inclinometros = df_gkn['Inclinómetro'].unique()
variables = ['A+', 'A-', 'B+', 'B-']

for inclino in inclinometros:
    df_vis = df_gkn[df_gkn['Inclinómetro'] == inclino]
    print(f"Visualizando inclinómetro: {inclino}")

    # a) Gráfico variable vs Profundidad, coloreado por fecha
    for var in variables:
        fig = px.scatter(
            df_vis,
            x='Profundidad',
            y=var,
            color='Fecha',
            title=f'{var} vs Profundidad para {inclino}',
            hover_data=['Inclinómetro', 'Fecha', 'Nro. Sonda']
        )
        fig.update_traces(marker=dict(size=8), selector=dict(mode='markers'))
        fig.show()

    # b) Gráfico de evolución temporal: variable vs Fecha (promedio por fecha/profundidad)
    for var in variables:
        df_agg = df_vis.groupby(['Fecha', 'Profundidad'])[var].mean().reset_index()
        fig = px.line(
            df_agg,
            x='Fecha',
            y=var,
            color='Profundidad',
            title=f'Evolución temporal de {var} por profundidad para {inclino}',
            markers=True
        )
        fig.show()

print("¡Visualizaciones interactivas generadas para todos los inclinómetros y variables!")

# Paso 6: Visualizaciones Interactivas y Análisis por Profundidad y Fecha

En este paso puedes:
- Analizar las variables (A+, A-, B+, B-) por profundidad y fecha para cada inclinómetro.
- Generar gráficos interactivos con Plotly para explorar tendencias y patrones.
- Descargar o guardar los gráficos como archivos HTML interactivos en tu PC, Colab o Google Drive.

**Tipos de gráficos incluidos:**
- Dispersión de cada variable vs Profundidad, coloreado por fecha.
- Evolución temporal de cada variable para cada profundidad.

Esto te permite detectar fácilmente anomalías, tendencias o cambios relevantes en las mediciones a lo largo del tiempo y la profundidad.

In [ ]:
# PASO 6: Visualizaciones interactivas de inclinómetros con Plotly (profundidad en eje Y)
#
# Gráficos interactivos para cada variable y cada inclinómetro.
# El eje Y es la profundidad (vertical), el eje X es la variable (A+, A-, B+, B-), coloreado por fecha.
# No se guardan los gráficos, solo se muestran.

import plotly.express as px
import pandas as pd

df_gkn['Fecha'] = pd.to_datetime(df_gkn['Fecha'], errors='coerce')

inclinometros = df_gkn['Inclinómetro'].unique()
variables = ['A+', 'A-', 'B+', 'B-']

for inclino in inclinometros:
    df_vis = df_gkn[df_gkn['Inclinómetro'] == inclino]
    print(f"Visualizando inclinómetro: {inclino}")

    # a) Gráfico variable vs Profundidad (profundidad en eje Y, invertido)
    for var in variables:
        fig = px.scatter(
            df_vis,
            x=var,
            y='Profundidad',
            color='Fecha',
            title=f'{var} vs Profundidad para {inclino}',
            hover_data=['Inclinómetro', 'Fecha', 'Nro. Sonda']
        )
        fig.update_traces(marker=dict(size=8), selector=dict(mode='markers'))
        fig.update_yaxes(autorange="reversed", title="Profundidad")  # Eje Y invertido (más profundo abajo)
        fig.update_xaxes(title=var)
        fig.show()

    # b) Gráfico de evolución temporal: variable vs Fecha (promedio por fecha/profundidad)
    for var in variables:
        df_agg = df_vis.groupby(['Fecha', 'Profundidad'])[var].mean().reset_index()
        fig = px.line(
            df_agg,
            x='Fecha',
            y='Profundidad',
            color=var,
            title=f'Profundidad vs Fecha coloreado por {var} para {inclino}',
            markers=True
        )
        fig.update_yaxes(autorange="reversed", title="Profundidad")
        fig.update_xaxes(title="Fecha")
        fig.show()

print("¡Visualizaciones interactivas generadas para todos los inclinómetros y variables con profundidad en eje Y!")

In [ ]:
# PASO 7: Guardar gráficos Plotly de A+ y B+ vs Profundidad en disco local, Colab o Google Drive
#
# Esta celda genera y guarda los gráficos interactivos de A+ y B+ vs Profundidad
# para cada inclinómetro. Puedes elegir dónde guardarlos: PC, /content o Google Drive.
#
# Los archivos se guardan en formato HTML interactivo.

import plotly.express as px
import plotly.io as pio
import os

# Asegura que la fecha sea tipo datetime
df_gkn['Fecha'] = pd.to_datetime(df_gkn['Fecha'], errors='coerce')

print("¿Dónde quieres guardar los gráficos?")
print("1 - Descargar a mi computadora (PC)")
print("2 - Guardar en /content de Colab")
print("3 - Guardar en Google Drive (asegúrate de montar)")
opcion = input("Elige 1, 2 o 3: ").strip()

if opcion == "3":
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    ruta_drive = input("Escribe la ruta completa en tu Drive donde guardar los archivos (ej: /content/drive/MyDrive/MisGKN): ").strip()
    os.makedirs(ruta_drive, exist_ok=True)
elif opcion == "2":
    ruta_content = "/content"
elif opcion == "1":
    pass
else:
    print("Opción no válida.")
    opcion = None

# Para cada inclinómetro, genera y guarda los gráficos
inclinometros = df_gkn['Inclinómetro'].unique()
variables = ['A+', 'B+']

for inclino in inclinometros:
    df_vis = df_gkn[df_gkn['Inclinómetro'] == inclino]
    for var in variables:
        fig = px.scatter(
            df_vis,
            x=var,
            y='Profundidad',
            color='Fecha',
            title=f'{var} vs Profundidad para {inclino}',
            hover_data=['Inclinómetro', 'Fecha', 'Nro. Sonda']
        )
        fig.update_traces(marker=dict(size=8), selector=dict(mode='markers'))
        fig.update_yaxes(autorange="reversed", title="Profundidad")
        fig.update_xaxes(title=var)
        nombre_archivo = f"{str(inclino).replace(' ', '_').replace('/', '-')}_{var}_vs_Profundidad.html"

        if opcion == "1":
            pio.write_html(fig, file=nombre_archivo)
            from google.colab import files
            files.download(nombre_archivo)
            print(f"Descargado: {nombre_archivo}")
        elif opcion == "2":
            ruta_html = os.path.join(ruta_content, nombre_archivo)
            pio.write_html(fig, file=ruta_html)
            print(f"Guardado en /content: {ruta_html}")
        elif opcion == "3":
            ruta_html = os.path.join(ruta_drive, nombre_archivo)
            pio.write_html(fig, file=ruta_html)
            print(f"Guardado en Google Drive: {ruta_html}")
        else:
            pass

print("¡Gráficos de A+ y B+ vs Profundidad guardados exitosamente!")